In [1]:
#6.5.2 Create Heatmaps for Weather Parameters

# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()



,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2021-10-01 01:39:35,-46.1927,168.8643,53.47,70,100,7.87
1,1,Esperance,AU,2021-10-01 01:39:36,-33.8667,121.9000,64.24,51,100,10.09
2,2,Barrow,US,2021-10-01 01:39:32,71.2906,-156.7887,30.22,74,90,10.36
3,3,Yellowknife,CA,2021-10-01 01:37:39,62.4560,-114.3525,48.42,100,75,8.05
4,4,Mahebourg,MU,2021-10-01 01:39:36,-20.4081,57.7000,71.89,78,75,12.66


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(g_key)

In [4]:
#Creating a heatmap of max temps

# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
#heatmap of humidity

# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#Cloudiness heatmap

locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df.Cloudiness
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#Wind speed heatmap

locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#6.5.3 Get Vacation Criteria

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [13]:
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Sao Filipe,CV,2021-10-01 01:39:37,14.8961,-24.4956,79.32,77,68,3.20
8,8,Faanui,PF,2021-10-01 01:36:22,-16.4833,-151.7500,79.12,73,2,17.31
9,9,Kapaa,US,2021-10-01 01:36:20,22.0752,-159.3190,84.18,67,20,3.00
10,10,Half Moon Bay,US,2021-10-01 01:39:38,37.4636,-122.4286,87.62,60,1,4.61
13,13,Natal,BR,2021-10-01 01:39:38,-5.7950,-35.2094,77.65,88,20,4.61


In [19]:
#6.5.4 Map Vacation Criteria

#Getting hotel data from the places API and create a hotel dataframe

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Sao Filipe,CV,79.32,14.8961,-24.4956,
8,Faanui,PF,79.12,-16.4833,-151.7500,
9,Kapaa,US,84.18,22.0752,-159.3190,
10,Half Moon Bay,US,87.62,37.4636,-122.4286,
13,Natal,BR,77.65,-5.7950,-35.2094,


In [20]:
#Google Places API request

#first we need to set the parameters for the requests.get call

params = {"radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [22]:
#Iterating through hotel_df and find hotels for each city

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print('Hotel not found.... Skipping')

Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping


In [35]:
hotel_df.head()


,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Sao Filipe,CV,79.32,14.8961,-24.4956,Hotel Xaguate
8,Faanui,PF,79.12,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
9,Kapaa,US,84.18,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Half Moon Bay,US,87.62,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
13,Natal,BR,77.65,-5.7950,-35.2094,Villa Park Hotel | Natal


In [24]:
#Create a max temp heat map for vacation spots (hotel_df)

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
#adding markers for each city on top of the heatmap

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
'''Now we can add a pop-up marker for each city that displays the hotel name, city name, country, and maximum temperature.'''


info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Xaguate</dd>\n<dt>City</dt><dd>Sao Filipe</dd>\n<dt>Country</dt><dd>CV</dd>\n<dt>Max Temp</dt><dd>79.32 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bora-Bora Pearl Beach Resort & Spa</dd>\n<dt>City</dt><dd>Faanui</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Max Temp</dt><dd>79.12 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sheraton Kauai Coconut Beach Resort</dd>\n<dt>City</dt><dd>Kapaa</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>84.18 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>The Ritz-Carlton, Half Moon Bay</dd>\n<dt>City</dt><dd>Half Moon Bay</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>87.62 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Villa Park Hotel | Natal</dd>\n<dt>City</dt><dd>Natal</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Max Temp</dt><dd>77.65 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Max Temp</dt><d

In [28]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))